In [2]:
import logging
import wikipedia
wikipedia.set_lang('en')

from gensim import corpora, models, similarities

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [4]:
import csv
list = []
with open('IA/validation_set.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=';')
    for row in spamreader:
        list.append(row)

In [27]:
def wiki_eval(row):
    wiki = []
    logging.info(row)

    ReponseCorrecte = 1
    Question = 2
    QuestionListe = [3,4,5,6]

    c = 0

    for i in QuestionListe:
        logging.info(i)

        Resultat = wikipedia.search(row[i], results=3)

        for m,j in enumerate(Resultat):

            logging.info(j)
            logging.info(m)

            try:
                z = wikipedia.summary(j)
            except wikipedia.exceptions.DisambiguationError as e:
                for k in [g for f,g in enumerate(e.options) if f <=3]:
                    try :
                        z = wikipedia.summary(k) 
                    except:
                        logging.error("top")
            logging.info(z)
            logging.info(c)
            c += 1
            wiki.append((i-3, row[i],z))
    
    return wiki

In [6]:
Result = []
Dict = {'A':0,
        'B':1,
        'C':2,
        'D':3,
       }

In [28]:
stoplist = set('for a of the and to in is not'.split())
for seq in [j for j in range(5)]:

    row = list[seq+1]
    wiki = wiki_eval(row)
    documents = [i[2] for i in wiki]
    texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

ERROR:root:top
ERROR:root:top


In [29]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

    texts_ = [[token for token in text if frequency[token] > 1]
         for text in texts]
from pprint import pprint   # pretty-printer
pprint(texts_)

[[u'heart',
  u'other',
  u'which',
  u'pumps',
  u'blood',
  u'through',
  u'blood',
  u'circulatory',
  u'blood',
  u'body',
  u'with',
  u'oxygen',
  u'also',
  u'metabolic',
  u'heart',
  u'humans,',
  u'other',
  u'heart',
  u'divided',
  u'into',
  u'left',
  u'right',
  u'lower',
  u'left',
  u'right',
  u'right',
  u'atrium',
  u'ventricle',
  u'are',
  u'as',
  u'right',
  u'heart',
  u'their',
  u'left',
  u'as',
  u'left',
  u'have',
  u'two',
  u'an',
  u'atrium',
  u'while',
  u'have',
  u'three',
  u'healthy',
  u'heart',
  u'blood',
  u'one',
  u'through',
  u'heart',
  u'heart',
  u'which',
  u'heart',
  u'which',
  u'also',
  u'small',
  u'amount',
  u'wall',
  u'heart',
  u'three',
  u'heart',
  u'pumps',
  u'blood',
  u'through',
  u'both',
  u'circulatory',
  u'blood',
  u'oxygen',
  u'from',
  u'systemic',
  u'right',
  u'atrium',
  u'from',
  u'passes',
  u'right',
  u'from',
  u'it',
  u'pumped',
  u'into',
  u'pulmonary',
  u'through',
  u'lungs',
  u'it',
  u'o

In [30]:
dictionary = corpora.Dictionary(texts_)
dictionary.save('/tmp/deerwester.dict')

In [31]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)

In [32]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=4)

In [33]:
index = similarities.MatrixSimilarity(lsi[corpus])

[u'0.519*"are" + 0.405*"fats" + 0.202*"fatty" + 0.176*"by" + 0.163*"as" + 0.157*"acid" + 0.143*"called" + 0.138*"from" + 0.136*"that" + 0.135*"can"',
 u'-0.532*"heart" + -0.436*"blood" + -0.235*"right" + -0.235*"left" + -0.201*"through" + -0.196*"from" + 0.186*"fats" + -0.134*"ventricle" + -0.134*"pumps" + -0.124*"oxygen"',
 u'0.532*"or" + 0.328*"access" + 0.219*"flush" + 0.210*"an" + 0.202*"with" + 0.202*"be" + 0.110*"name" + 0.109*"need" + 0.109*"door" + 0.109*"applicable."',
 u'-0.540*"receptor" + -0.479*"glucagon-like" + -0.417*"peptide" + -0.221*"glucagon" + -0.195*"receptors" + -0.154*"1" + -0.154*"binds" + -0.154*"2" + -0.126*"inhibitory" + -0.126*"related"']

In [45]:
doc = list[1][2]
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print vec_lsi

[(0, 0.83655272611147313), (1, 0.15395213049829568), (2, -0.13220009286918247), (3, -0.035133834320255256)]


In [53]:
doc = wikipedia.summary('yard')

In [54]:
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print vec_lsi

[(0, 1.5642503125225837), (1, -0.67509370309609829), (2, 0.93365557169615965), (3, -0.25159803763824246)]


In [56]:
wiki = wiki_eval(list[1])


ERROR:root:top


[(0, 'cool foods during shipping', u'Freezing food preserves it from the time it is prepared to the time it is eaten. Since early times, farmers, fishermen, and trappers have preserved their game and produce in unheated buildings during the winter season. Freezing food slows down decomposition by turning residual moisture into ice, inhibiting the growth of most bacterial species. In the food commodity industry, there are two processes: mechanical and cryogenic (or flash freezing). The freezing kinetics is important to preserve the food quality and texture. Quicker freezing generates smaller ice crystals and maintains cellular structure. Cryogenic freezing is the quickest freezing technology available due to the ultra low liquid nitrogen temperature \u2212196 \xb0C (\u2212320.8 \xb0F)}.\nPreserving food in domestic kitchens during the 20th and 21st centuries is achieved using household freezers. Accepted advice to householders was to freeze food on the day of purchase. An initiative by 

In [66]:
R = []
for i in wiki:
    doc = i[2]
    vec_bow = dictionary.doc2bow(doc.lower().split())
    vec_lsi = lsi[vec_bow] # convert the query to LSI space
    R.append((i[0], i[1], vec_lsi[0]))

In [67]:
R

[(0, 'cool foods during shipping', (0, 2.2736013233737391)),
 (0, 'cool foods during shipping', (0, 4.2937683069481336)),
 (0, 'cool foods during shipping', (0, 1.0606926337380056)),
 (1, 'yard', (0, 1.5642503125225837)),
 (1, 'yard', (0, 6.8450316628409249)),
 (1, 'yard', (0, 1.5754162384288806)),
 (2, 'nervous system', (0, 15.127500139808079)),
 (2, 'nervous system', (0, 0.0070522582233138098)),
 (2, 'nervous system', (0, 0.39961117677002078)),
 (3, 'birds of prey', (0, 5.8646140405900917)),
 (3, 'birds of prey', (0, 0.85694120959550268)),
 (3, 'birds of prey', (0, 0.25187645382047452))]